* Criado por: Gabriel Quintella
* Data Criação: 24.01.2023
* Data Última Modificação: 30.01.2023
* Repositório: https://github.com/dbaassists/Projeto-KB-Spark/blob/main/Projeto%20KB%20Spark.ipynb

DESCRIÇÃO DO PROJETO

Criar uma base de conhecimento (KB) com comandos pyspark.

*  Dia 1 - 24.01.2023 - Lançamento do Projeto;
*  Dia 2 - 26.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 3 - 27.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 4 - 28.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 5 - 29.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;
*  Dia 6 - 30.01.2023 - Aprimoramento do Projeto com melhoria nos exemplos criados;


 Sumário

* 01 - Instalação da Biblioteca PySpark
* 02 - Definição de Bibliotecas
* 03 - Métodos de Importação de Arquivos CSV
* 04 - Transformando o DataFrame Spark em Tabela Temporária
* 05 - Consultando a Estrutura do DataFrame Spark
* 06 - Lendo a Tabela e Transformando em outro DataFrame Spark
* 07 - Convertendo o DataType de uma Coluna de um DataFrame Spark
* 08 - Convertendo o DataType e Criando uma nova Coluna de um DataFrame Spark
* 09 - Renomeando uma Coluna em DataFrame Spark
* 10 - Renomeando Várias Colunas em DataFrame Spark
* 11 - Ordenando um DataFrame Spark


# 01 - Instalação da Biblioteca PySpark

In [ ]:
#pip install pyspark

# 02 - Definição de Bibliotecas

In [ ]:
import sys
import requests
import pandas as pd
import json
import pyspark.sql.functions as F #import explode_outer, col
from pyspark.sql.types import *
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkConf
from pyspark import SparkContext
import urllib
from sqlalchemy import __version__ as sa_version, create_engine, text


# 03 - Métodos de Importação de Arquivos CSV

* A CSV dataset is pointed to by path.
* The path can be either a single CSV file or a directory of CSV files
* path = "examples/src/main/resources/people.csv"

df = spark.read.csv(path)

df.show()

# 03.1 - Importando Arquivo CSV de forma FULL

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

df = spark.read.csv(arquivo)

df.show(2,truncate=False)

# 03.2 - Importando Arquivo CSV Definindo o Delimitador de Colunas

* Read a csv with delimiter, the default delimiter is ","

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

df = spark.read.option('delimiter',';').csv(arquivo)

#df.select('DT_GERACAO').show()

df.show(2,truncate=False)

In [137]:
#df.write.option("header",True) \
# .csv("/tmp/spark_output/zipcodes")

Py4JJavaError: An error occurred while calling o1355.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:651)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	

# Entendendo a diferença entre as funções option() e options()

# 03.3 - Importando Arquivo CSV Definindo Delimitador de Colunas e Cabeçalho

## Usando Função: option()

* Read a csv with delimiter and a header

In [139]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

df = spark.read.option('delimiter',';') \
                .option('header','True') \
                .csv(arquivo)

#df.select('DT_GERACAO').show()

df.show(2,truncate=False)

+----------+----------+-----------+---------------+-----------------+--------+----------+------------------------------+----------+--------------+-----+-----+--------------+--------+-----------------+------------+------------+---------------------+---------------------+-------------------+----------------+--------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+------------------------------+------------+-------------------+------------+-----------------------+------------+---------------+-----------------------+----------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------------+-----------------------------+---------+---------+-----------------+-------------------+---------------+---------------+-----------+-----------+-----------+-------------------------+-----------------------+----------------+----------------+------------

## Usando Função: options()

* You can also use options() to use multiple options

In [141]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

df = spark.read.options(delimiter=';'
                        , header='True') \
                        .csv(arquivo)

#df.select('DT_GERACAO').show()

df.show(2,truncate=False)

+----------+----------+-----------+---------------+-----------------+--------+----------+------------------------------+----------+--------------+-----+-----+--------------+--------+-----------------+------------+------------+---------------------+---------------------+-------------------+----------------+--------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+------------------------------+------------+-------------------+------------+-----------------------+------------+---------------+-----------------------+----------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------------+-----------------------------+---------+---------+-----------------+-------------------+---------------+---------------+-----------+-----------+-----------+-------------------------+-----------------------+----------------+----------------+------------

# 03.4 - Importando TODOS os Arquivos CSV de um Diretório

## Usando spark.read.csv()

* Read all files in a folder, please make sure only CSV files should present in the folder.

In [ ]:

folderPath = "../ProjetoSparkKB/Dados/Candidato/"
df5 = spark.read.csv(folderPath,sep=';')
df5.count()

## Usando spark.read.format()


In [ ]:
arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo2 = spark.read.format("csv").load(arquivo)
    
dfExemplo2.printSchema()

# 03.5 - Importando Arquivo CSV Definindo inferSchema

## Entendendo o parâmetro inferSchema

## Parâmetro inferSchema FALSE

In [ ]:
arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo3 = spark.read.option("inferSchema",False) \
                .option("delimiter",";") \
                .option("header",True) \
                .csv(arquivo)

dfExemplo3.printSchema()

## Parâmetro inferSchema TRUE

In [142]:
arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo04 = spark.read.options(header='True'
                                 , inferSchema='True'
                                 , delimiter=';') \
                                 .csv(arquivo)

dfExemplo04.printSchema()

root
 |-- DT_GERACAO: string (nullable = true)
 |-- HH_GERACAO: timestamp (nullable = true)
 |-- ANO_ELEICAO: integer (nullable = true)
 |-- CD_TIPO_ELEICAO: integer (nullable = true)
 |-- NM_TIPO_ELEICAO: string (nullable = true)
 |-- NR_TURNO: integer (nullable = true)
 |-- CD_ELEICAO: integer (nullable = true)
 |-- DS_ELEICAO: string (nullable = true)
 |-- DT_ELEICAO: string (nullable = true)
 |-- TP_ABRANGENCIA: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- SG_UE: string (nullable = true)
 |-- NM_UE: string (nullable = true)
 |-- CD_CARGO: integer (nullable = true)
 |-- DS_CARGO: string (nullable = true)
 |-- SQ_CANDIDATO: long (nullable = true)
 |-- NR_CANDIDATO: integer (nullable = true)
 |-- NM_CANDIDATO: string (nullable = true)
 |-- NM_URNA_CANDIDATO: string (nullable = true)
 |-- NM_SOCIAL_CANDIDATO: string (nullable = true)
 |-- NR_CPF_CANDIDATO: long (nullable = true)
 |-- NM_EMAIL: string (nullable = true)
 |-- CD_SITUACAO_CANDIDATURA: integer (nullabl

# 03.6 - Importando Arquivo CSV Definindo encoding

## Entendendo o parâmetro encoding

In [145]:

arquivo = '../ProjetoSparkKB/Dados/Candidato/consulta_cand_2022_BRASIL.csv'

dfExemplo04 = spark.read.options(header='True'
                                 , inferSchema='True'
                                 , delimiter=';'
                                 , encoding="windows-1252") \
                                 .csv(arquivo)

dfExemplo04.show()


+----------+-------------------+-----------+---------------+-----------------+--------+----------+--------------------+----------+--------------+-----+-----+-----------------+--------+-----------------+------------+------------+--------------------+--------------------+-------------------+----------------+--------------+-----------------------+-----------------------+------------------------+------------------------+---------------+----------+-------------+--------------------+------------+--------------------+--------------+-----------------------+------------+---------------+-----------------------+----------------+----------------+----------------+-----------------------+-----------------------+-------------+-------------------+-----------------------------+---------+---------+-----------------+--------------------+---------------+---------------+-----------+-----------+-----------+--------------------+-----------------------+----------------+----------------+------------+----------

# 03.7 - Definindo a Estrutura de um DataFrame e Importar CSV

** Documentação: https://sqlrelease.com/spark-read-file-with-special-characters-using-pyspark

** Documentação: https://sparkbyexamples.com/pyspark/pyspark-sql-types-datatype-with-examples/

** Documentação: https://spark.apache.org/docs/latest/sql-ref-datatypes.html

** Documentação: https://sparkbyexamples.com/pyspark/pyspark-date_format-convert-date-to-string-format/

### O arquivo "indicador_homocidio.csv" foi criado com a coluna Data do tipo "Data"
### O que permitiu a criação do DataFrame com a coluna Data do tipo DateType()

In [146]:
arquivo = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio.csv'


schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",DateType(),True) \


df_with_schema = spark.read.options(header='True'
                                    , inferSchema='False'
                                    , delimiter=';'
                                    , encoding="windows-1252") \
                                    .schema(schema) \
                                    .csv(arquivo)

df_with_schema.printSchema()

df_with_schema.show()

root
 |-- CodUf: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Quantidade: integer (nullable = true)
 |-- Data: date (nullable = true)

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|2023-01-28|
|              Amapá|Homicídio doloso|      1948|2023-01-29|
|               Acre|Homicídio doloso|      2007|2023-01-30|
|          Tocantins|Homicídio doloso|      2787|2023-01-31|
|           Rondônia|Homicídio doloso|      3482|2023-02-01|
|   Distrito Federal|Homicídio doloso|      3518|2023-02-02|
| Mato Grosso do Sul|Homicídio doloso|      3905|2023-02-03|
|              Piauí|Homicídio doloso|      4714|2023-02-04|
|     Santa Catarina|Homicídio doloso|      5832|2023-02-05|
|        Mato Grosso|Homicídio doloso|      6920|2023-02-06|
|            Sergipe|Homicídio doloso|      69

# 04 - Trabalhando com Datas - Converte coluna String (DD/MM/AAA) para Date (AAAAMMDD)

### O arquivo "indicador_homocidio_2.csv" foi criado com a coluna Data do tipo "Geral"
### Nesse caso, vamos precisar criar o DataFrame com a coluna Data do tipo StringType() e em seguida realizar a conversão da coluna para o tipo Date

* Trabalhando com Datas

** Documentação: https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

In [150]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType

arquivo = '../ProjetoSparkKB/Dados/IndicadorSegurancaPublica/indicador_homocidio_2.csv'

schema = StructType() \
      .add("CodUf",StringType(),True) \
      .add("TipoCrime",StringType(),True) \
      .add("Quantidade",IntegerType(),True) \
      .add("Data",StringType(),True) \

df_with_schema2 = spark.read.options(header='True', inferSchema='False', delimiter=';', encoding="windows-1252") \
      .schema(schema) \
      .csv(arquivo)

df_with_schema2.show()

print('------------------------------ ESTRUTURA ANTES DA CONVERSÃO ------------------------------')

df_with_schema2.printSchema()

print('------------------------------------------------------------------------------------------')

df_with_schema2 = df_with_schema2.withColumn('Data', to_date(date_format(unix_timestamp(df_with_schema2.Data , "dd/mm/yyyy").cast("timestamp"),"yyyy-mm-dd")))

print('------------------------------ ESTRUTURA DEPOIS DA CONVERSÃO ------------------------------')

df_with_schema2.printSchema()

df_with_schema2.show()

+-------------------+----------------+----------+----------+
|              CodUf|       TipoCrime|Quantidade|      Data|
+-------------------+----------------+----------+----------+
|            Roraima|Homicídio doloso|      1389|28/01/2023|
|              Amapá|Homicídio doloso|      1948|29/01/2023|
|               Acre|Homicídio doloso|      2007|30/01/2023|
|          Tocantins|Homicídio doloso|      2787|31/01/2023|
|           Rondônia|Homicídio doloso|      3482|01/02/2023|
|   Distrito Federal|Homicídio doloso|      3518|02/02/2023|
| Mato Grosso do Sul|Homicídio doloso|      3905|03/02/2023|
|              Piauí|Homicídio doloso|      4714|04/02/2023|
|     Santa Catarina|Homicídio doloso|      5832|05/02/2023|
|        Mato Grosso|Homicídio doloso|      6920|06/02/2023|
|            Sergipe|Homicídio doloso|      6970|07/02/2023|
|           Amazonas|Homicídio doloso|      8590|08/02/2023|
|     Espírito Santo|Homicídio doloso|      8695|09/02/2023|
|            Paraíba|Hom

# 04.1 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Ano

In [ ]:

df_with_schema2 = df_with_schema2.withColumn("Ano", date_format(col("Data"), "y"))


df_with_schema2.printSchema()

# 04.2 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Mês

In [ ]:

df_with_schema2 = df_with_schema2.withColumn("NumMes", date_format(col("Data"), "M"))

df_with_schema2 = df_with_schema2.withColumn("NomMes", date_format(col("Data"), "L"))


df_with_schema2.printSchema()

df_with_schema2.show()

# 04.3 - Trabalhando com Datas - A partir de uma coluna de Data, criar outra de Dia

In [ ]:

df_with_schema2 = df_with_schema2.withColumn("NumDiaAno", date_format(col("Data"), "D"))

df_with_schema2 = df_with_schema2.withColumn("NumDiaMes", date_format(col("Data"), "d"))

df_with_schema2.printSchema()

df_with_schema2.show()

# 05 - Trabalhando com Arquivos Parquet

In [ ]:
#arquivoDestino = '../ProjetoSparkKB/Dados/Parquet/indicador_homocidio.parquet'

#df_with_schema3.write.mode("overwrite").parquet(arquivoDestino)

# 06 - Comando Semelhante ao DataFrame.info() do Pandas

In [ ]:
df_with_schema3.summary().show()

# 04 - Transformando o DataFrame Spark em Tabela Temporária

In [ ]:
df.createOrReplaceTempView('teste')

# 05 - Consultando a Estrutura do DataFrame Spark

In [ ]:
df.printSchema()

# 06 - Lendo a Tabela e Transformando em outro DataFrame Spark

In [ ]:
df2 = spark.sql("""
SELECT DT_GERACAO 
FROM TESTE
""")

# 07 - Convertendo o DataType de uma Coluna de um DataFrame Spark

In [ ]:
df2.withColumn("DT_GERACAO", df2["DT_GERACAO"].cast("timestamp")).printSchema()

# 08 - Convertendo o DataType e Criando uma nova Coluna de um DataFrame Spark

In [ ]:
df2.withColumn("DTGERACAO", df2["DT_GERACAO"].cast("timestamp")).printSchema()

# 09 - Renomeando uma Coluna em DataFrame Spark

In [ ]:
df2.withColumnRenamed("DT_GERACAO","DT_GERACAO2").printSchema()

# 10 - Renomeando Várias Colunas em DataFrame Spark

In [ ]:
df3 = spark.sql("""
SELECT DT_GERACAO , HH_GERACAO
FROM TESTE
""")

In [ ]:
df3 = df3.withColumnRenamed("DT_GERACAO","DtGeracao") \
    .withColumnRenamed("HH_GERACAO","HrGeracao")
df3.printSchema()

# 11 - Ordenando um DataFrame Spark

In [ ]:
df4 = spark.sql("""
SELECT SG_UF , NM_URNA_CANDIDATO
FROM TESTE
""")

In [ ]:
df4.show()

In [ ]:
df4.sort("SG_UF").show(truncate=False)


# 11.1 - Ordenando um DataFrame Spark por Várias Colunas

In [ ]:
df4.sort("SG_UF","NM_URNA_CANDIDATO").show(truncate=False)


In [ ]:

df4.sort(col("SG_UF"),col("NM_URNA_CANDIDATO")).show(truncate=False)

# 12 - Agrupando um DataFrame Spark

In [ ]:
df4.groupBy("SG_UF").count().show(truncate=False)

In [ ]:
df5 = spark.sql("""
SELECT SG_UF , NM_URNA_CANDIDATO, SQ_CANDIDATO
FROM TESTE
""")

# 13 - Funções de Agrupamento (Count(), Sum(), Min(), Max() e Mean())

In [ ]:
df5.groupBy("SG_UF") \
    .agg(count("SQ_CANDIDATO").alias("count_SQ_CANDIDATO"), \
         sum("SQ_CANDIDATO").alias("sum_SQ_CANDIDATO"), \
         avg("SQ_CANDIDATO").alias("avg_SQ_CANDIDATO"), \
         min("SQ_CANDIDATO").alias("min_SQ_CANDIDATO"), \
         max("SQ_CANDIDATO").alias("max_SQ_CANDIDATO") \
     ) \
    .show(truncate=False)